In [3]:
#Dictionary matching up case numbers and study ID

# Lists of integers and strings
case_numbers = [1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 23, 27, 37, 87, 88, 89, 90, 91, 93, 94, 95, 22, 24, 28, 29, 30, 32, 34, 36, 38, 39, 42, 43, 44, 46, 47, 48, 49, 50, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 81, 82, 83, 84, 86]

study_IDs = ["MESAL-5010098-E3", "MESAL-5010730-E3", "MESAL-6011969-E3", "MESAL-5010942-E3", "MESAL-7013450-E3", "MESAL-5010845-E3", "MESAL-8014868-E3", "MESAL-3016340-E3", "MESAL-3016587-E3", "MESAL-7018380-E3", "MESAL-6011020-E3", "MESAL-7018347-E3", "MESAL-5014832-E3", "MESAL-3012603-E3", "MESAL-6022812-E3", "MESAL-3012840-E3", "MESAL-7015569-E3", "MESAL-7018860-E3", "MESAL-3015246-E3", "MESAL-6011594-E3", "MESAL-3016900-E3", "MESAL-7012705-E3", "MESAL-6010377-E3", "MESAL-4021096-E3", "MESAL-5018196-E3", "MESAL-3021556-E3", "MESAL-8024995-E3", "MESAL-6015034-E3", "MESAL-4016645-E3", "SPI-JH110088", "SPI-JH110704", "SPI-JH112953", "SPI-JH115247", "SPI-JH114663", "SPI-JH110280", "SPI-JH115627", "SPI-JH112358", "SPI-IA210599", "SPI-JH112648", "SPI-IA210257", "SPI-IA210637", "SPI-IA210520", "SPI-JH114016", "SPI-IA210102", "SPI-JH115778", "SPI-IA210428", "SPI-IA210305", "SPI-IA210273", "SPI-JH114615", "SPI-JH111728", "SPI-IA210196", "SPI-IA210285", "SPI-MU163450", "SPI-JH115593", "SPI-JH111894", "SPI-JH112757", "SPI-MU164552", "SPI-MU164261", "SPI-JH114008", "SPI-JH113081", "SPI-JH114158", "SPI-JH111913", "SPI-JH114303", "SPI-JH112683", "SPI-IA210545", "SPI-JH115746", "SPI-JH114090", "SPI-JH111112", "SPI-JH111791", "SPI-JH114588", "SPI-IA210649", "SPI-IA210581", "SPI-JH112431", "SPI-JH112455", "SPI-JH112708", "SPI-JH114311", "SPI-MU161671"]

# Creating the dictionary
number_to_ID = dict(zip(case_numbers, study_IDs)) 
ID_to_number = dict(zip(study_IDs, case_numbers))

In [2]:
number_to_ID[22]

'SPI-JH110088'

In [2]:
import itk
#Test case files to ensure they load properly.
testpath = "Y:/MESA_V5--ATS-VessDys-TLCs/1-UseThis-Unzipped-APKs/H-10983_7018860_MESA-7018860-E5_CHEST_0.75_B35f_25023450/ZUNU_vida-lung.hdr"
testread = itk.imread(testpath, itk.SS)
if testread:
    print("Itk read sucessful")

Itk read sucessful


In [14]:
# Function for finding the lung mask header file corresponding to a study when given the study ID. You only need to know the
# case number that you would like to use, then you can use the number_to_ID dictionary to retrieve the corresponding studyID to
# pass into this function.
# RETURNS this function returns the full path to the header file that will be read in.

import glob
import os
def find_header_file(studyID):
    # Specify the main folder path
    if "SPI" in studyID:
        main_folder_path = "Y:/SPIR--ATS-VessDys-TLCs/1-UseThis-Unzipped-APKs"  # Replace with the actual path to your main folder
        subsub = True #Spiromics files will have two subfolders to go into. Mesal only has one.
    elif "MESAL" in studyID:
        main_folder_path = "Y:/MESA_V5--ATS-VessDys-TLCs/1-UseThis-Unzipped-APKs"
        mesalID = studyID
        mesalID = mesalID[0:-3]   #Some mesal files are listed as MESAL instead of MESA.
        studyID = studyID.replace("L", "")   #Fixes issue where Mesal files are shown as just "MESA" instead of "MESAL" in these subfolders
        studyID = studyID[0:-3]   #Fixes E3 vs E5 issue
        subsub = False

    # Use glob to find subfolders matching the given part of the name.
    # Subfolder name is repeated twice so need to account for that and store as subsubfolders.
    subfolders = glob.glob(os.path.join(main_folder_path, f'*{studyID}*'))

    if not subfolders:
        subfolders = glob.glob(os.path.join(main_folder_path, f'*{mesalID}*'))
        if not subfolders:
            print(f"No subfolder found with '{studyID}' in the name.")
            return None
    
    if not subsub:
        subfolders[0] = subfolders[0].replace("\\", "/")
        # Assume there's only one subfolder with the given part in its name
        target_subfolder = subfolders[0]
    if subsub:
        subsubfolders = glob.glob(os.path.join(subfolders[0], f'*{studyID}*'))
        subsubfolders[0] = subsubfolders[0].replace("\\", "/")
        target_subfolder = subsubfolders[0]

    # Find the header file in the target subfolder
    header_file_path = os.path.join(target_subfolder, 'ZUNU_vida-lung.hdr')
    header_file_path = header_file_path.replace("\\", "/")
    
    # Check if the header file exists
    if os.path.exists(header_file_path):
        return header_file_path
    else:
        print(f"No 'ZUNU_vida-lung.hdr' file found in the subfolder.")
        return None

In [15]:
# Example: Find the header file for a subfolder containing 'studyID' in its name
studyID = number_to_ID[4]
header_file_path = find_header_file(studyID)
print(f"Looking at study ID {studyID}")
if "SPI" in studyID:
    print("This is a Spiromics file.")
elif "MESAL" in studyID:
    print("This is a Mesal file.")
if header_file_path:
    print(f"Header file found at: {header_file_path}")
else:
    print("Header file not found.")

Looking at study ID MESAL-5010942-E3
This is a Mesal file.
Header file found at: Y:/MESA_V5--ATS-VessDys-TLCs/1-UseThis-Unzipped-APKs/H-11740_5010942_MESAL-5010942-E3_TLC_0.75_Qr40d-5_54830375/ZUNU_vida-lung.hdr


In [35]:
number_to_ID[1]

'MESAL-5010098-E3'

In [28]:
#Use for MESAL files to accurately print out the correct study ID into the word doc.
def extract_studyID(input_string, keyword):
    if "ESA" in keyword:
        hold = keyword[0:-3]
        keyword = keyword[0:-3].replace("L", "")
        index = input_string.find(keyword)
    else:
        return keyword
    if index != -1:
        # If the keyword is found in the string
        result = input_string[index:index + len(keyword) + 3]
        return result
    else:
        # If the keyword is not found in the string
        index2 = input_string.find(hold)
        result = input_string[index2:index2 + len(hold) + 3]
        if index2 != -1:
            result = input_string[index2:index2 + len(hold) + 3]
            return result
        else:
            return None

# Example usage:
input_string = "Y:/MESA_V5--ATS-VessDys-TLCs/1-UseThis-Unzipped-APKs/H-3412_5010098_MESA-5010098-E5_CHEST_0.75_B35f_19310476/ZUNU_vida-lung.hdr"
result = extract_studyID(input_string, number_to_ID[1])
print(number_to_ID[1][0:-3].replace("L", ""))
if result:
    print("Result:", result)
else:
    print("Keyword not found in the string.")

MESA-5010098
Result: MESA-5010098-E5


In [29]:
import itk
import numpy as np
from mayavi import mlab

# Specify the file paths for the header and image data
# header_file_path = "Z:/Q- Figures&Papers/1-T32-Poster/Nick/Unzipped-Cases/H-14479_JH110088_SPI-JH110088-V2_TLC_0.75_B35f_23390683/H-14479_JH110088_SPI-JH110088-V2_TLC_0.75_B35f_23390683/ZUNU_vida-lung.hdr"
# unzipped_folder = "//lc-rs-store25.hpc.uiowa.edu/teamwork/public_resources/Scan_Exchange/for_Junfeng/SPIR--ATS-VessDys-TLCs/1-UseThis-Unzipped-APKs"
# image_file_path = "Z:/Q- Figures&Papers/1-T32-Poster/Nick/Unzipped-Cases/H-14479_JH110088_SPI-JH110088-V2_TLC_0.75_B35f_23390683/H-14479_JH110088_SPI-JH110088-V2_TLC_0.75_B35f_23390683/ZUNU_vida-lung.img.gz"
CaseNum = 4
header_file_path = find_header_file(number_to_ID[CaseNum])


# Read the image data using the information from the header
image = itk.imread(header_file_path, itk.SS)  # Adjust the pixel type (itk.SS for signed short) based on your image data type

# Get the image attributes
image_array = itk.array_from_image(image)
origin1 = image.GetOrigin()
spacing1 = image.GetSpacing()
size1 = image.GetLargestPossibleRegion().GetSize()
direction1 = image.GetDirection()

# Set values above zero to one
image_array[image_array > 0] = 1
# image_array = image_array[::10,::10,::10]
# Create a Mayavi scene
mlab.figure(size=(600, 600), bgcolor=(1, 1, 1))

#Center of mass coordinates
coords = np.argwhere(image_array)
center_of_mass = np.mean(coords, axis=0)
center_of_mass = center_of_mass.astype(int)

# Create a Mayavi volume with a single contour level at 0.9
mlab.contour3d(image_array, contours=[0.5], opacity=0.15, color=(128/255, 174/255, 128/255))

# Set the voxel spacing and origin
mlab.view(azimuth=90, elevation=0, distance=(700/0.546875)/10, focalpoint=center_of_mass)
distance = np.array(spacing1) * np.array(size1)
center = np.array(origin1) + np.array(distance) / 2
x, y, z = center[0], center[1], center[2]
ypt = y + 700/.546875
# distance = (np.array(spacing1) * np.array(size1))/10
# center = (np.array(origin1) + np.array(distance) / 2)
# x, y, z = center[0], center[1], center[2]
# ypt = y + 70/.546875 + 10
# camera_focal_point = np.array([x, y, z])   #From Nate's code, currently not needed for this specific case. Maybe for others?
camera_position = np.array([x, ypt, z])
mlab.gcf().scene.camera.position = camera_position
# mlab.gcf().scene.camera.focal_point = camera_focal_point   #From Nate's code ^^^
# mlab.gcf().scene.camera.view_up = [0, 0, 1]
mlab.gcf().scene.camera.clipping_range = (1, 10000)
mlab.gcf().scene.camera.azimuth(180)   #Added in. Needed due to the fact that this image is loaded in upside down?
mlab.gcf().scene.camera.elevation(180)

# Save a snapshot of the current Mayavi window to the specified folder path
studyID = header_file_path.split('/')[-2].split('_')[2].replace("-V2", "")
folder_path = "Z:/Q- Figures&Papers/1-T32-Poster/Nick/TLC-Images"
file_name = f"Case-{CaseNum}_{extract_studyID(header_file_path, number_to_ID[CaseNum])}_(dist700).png"  # You can change the file format as needed, e.g., "snapshot.png", "snapshot.jpg", etc.

mlab.savefig(folder_path + "/" + file_name)

# Display the plot
mlab.show()

In [18]:
#1 Gold Standard to write images to word document report.

# NOTES TO RUN: (MAKE SURE TO READ DIRECTIONS OR CONTACT NATE)
# 1- This code assumes you have already generated your images and placed them in a folder or folders 
# 2- Nate's Images were created in previous code. All Images files here Q- Figures&Papers/1-T32-Poster/Nate/ImageCreationFile/
# 3- For QC images ----> /Q- Figures&Papers/1-T32-Poster/Nate/ImageCreationFile/QC-Images
# 4- For Segmentation (A/V) Images and Aseed/LungMask Images ----> /Q- Figures&Papers/1-T32-Poster/Nate/ImageCreationFile/3D Images
# 5- Once you have created all images that you need, you will need to add the following variables:
#   5a- airway_folder = (LOCATION OF AIRWAY IMAGES)
#   5b- airway_files = glob.glob(airway_folder + '/*.png') This is to store all airway files from the folder 
# 6- Then Update the for loop variables according to convention below (could ask Chat GPT if you are struggling or message Nate)
#   6a- for row_index, (image_path, lung_mask_path, model_image_path,) in enumerate(zip(image_files, lung_mask_files, model_image_files), start=1):   
# 7- Open the images
# 8- Set Widths for Images
# 9- Then Add the run to add the images --> airways_cells = row_cells[5] .....
from docx import Document
import glob
from PIL import Image
from docx.shared import Inches, Cm, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ALIGN_VERTICAL

# Create a new Document object which will be used to build the Word document
doc = Document()

for section in doc.sections:
    section.page_width = Inches(16.5)
    # You may also want to adjust the margins if necessary
    section.left_margin = Inches(1)
    section.right_margin = Inches(1)

# Define the number of rows and columns for the table in the document
num_rows = 4
num_cols = 3
# Add a table to the document with the specified number of rows and columns
table = doc.add_table(rows=num_rows, cols=num_cols)
# Set a style for the table, which affects its visual layout
table.style = 'Table Grid'
row_height = Pt(90)  # Set the row height to however tall you want
# Iterate over each row in the table and set the height
for row in table.rows:
    row.height = row_height
# Define column titles for the table's header row
titles = ["Case Number", "Study ID", "Disease Maps"]
# Get the first row of the table which will serve as the header
header_cells = table.rows[0].cells

# Assign the defined titles to the header cells in the table
for i in range(num_cols):
    header_cells[i].text = titles[i]
    header_cells[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    header_cells[i].vertical_alignment = WD_ALIGN_VERTICAL.CENTER

# Define the paths to the folders containing the images
disease_map_folder = 'Z:/Q- Figures&Papers/1-T32-Poster/Nick/TLC-Images'

# Use glob to find all .png files in the specified directories. This stores the images to a variable based on their endings
disease_map_files = glob.glob(disease_map_folder + "/*.png*", recursive=True)

# Print to console as a simple progress indicator
print('Going into loop')


# Iterate over the zipped lists of image paths, starting with index 1
for row_index, (disease_map_path) in enumerate(zip(disease_map_files), start=1):                  

    # Access the cells of the current row in the table
    row_cells = table.rows[row_index].cells
    
    # Write the row index and case number to their respective cells and center the text
    study = disease_map_files[row_index - 1].split('/')[-1].split('_')[1]
    if "MES" in study:
        study = study[0:4] + 'L' + study[4:-1]
        study = study + "3"
    row_cells[0].text = str(ID_to_number[study])
    row_cells[1].text = disease_map_files[row_index - 1].split('/')[-1].split('_')[1]

    for idx in range(2):
        cell = row_cells[idx]
        cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph = cell.paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Open the disease map image file
    img = Image.open(disease_map_files[row_index-1])
    
    # Set the new width for the disease map image to be inserted into the document
    new_width = int(img.width * 1.7)

    # Insert the disease map image to the cell
    cell = row_cells[2]
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    cell_paragraph = cell.paragraphs[0]
    cell_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run = cell_paragraph.add_run()
    run.add_picture(disease_map_files[row_index-1], width=Cm(new_width/300))

# Save the document to the specified path
output_path = 'Z:/Q- Figures&Papers/1-T32-Poster/Nick/CaseImages.docx'
doc.save(output_path)
print(f"Document saved to: {output_path}")


Going into loop


IndexError: list index out of range

In [70]:
disease_map_files

['Z:/Q- Figures&Papers/1-T32-Poster/Nick/TLC-Images\\Case-1_MESAL-5010098-E3_(dist700).png']

In [ ]:
for section in doc.sections:
    section.page_width = Inches(16.5)
    # You may also want to adjust the margins if necessary
    section.left_margin = Inches(1)
    section.right_margin = Inches(1)